In [1]:
import numpy as np
import math

In [2]:
from cardsample_problem import *

|S| = 909
|O| = 539


In [3]:
T = P.m*P.n
init_true_state= get_random_state()
#init_belief = init_belief_particle(all_states)
init_belief_hist = init_belief_particle_s0(all_states)
card_problem = CardProblem(init_true_state, init_belief_hist)

card_problem.agent.tree = None

print("*** Testing POMCP ***")

pomcp = pomdp_py.POMCP(max_depth=6, discount_factor=1.,
                           num_sims=20000, exploration_const=20,
                           rollout_policy=card_problem.agent.policy_model,
                           num_visits_init=1)

n_iter = 2
rewards = np.zeros(n_iter)

for it in range(n_iter):
    card_problem.agent.set_belief(init_belief_particle_s0(all_states), prior = True)

    r, tree_i = test_planner(card_problem, pomcp, nsteps=T)
    print(card_problem.agent.tree, tree_i)

    rewards[it] = r

    card_problem.agent.tree = tree_i

print("average reward ", rewards.mean(), "over %d iter"%n_iter)

*** Testing POMCP ***
s0  State((1, 0, 0, 0, 0, 0, 0, 0, 0), 0, None)
==== Step 1 ====
True state: State((1, 0, 0, 0, 0, 0, 0, 0, 0), 0, None), 1 0
Action: Action(2)
Observation: Observation((0, 0, 0, 0, 0, 1)),  1
Reward: 0.0
True next state: State((2, 0, 0, 0, 0, 0, 0, 1, 0), 0, 0), 2 0
{Observation((0, 0, 0, 0, 1, 0)): VNode(2410.000, 0.000, 2409 | dict_keys([Action(0), Action(1), Action(2)])), Observation((0, 0, 0, 0, 0, 1)): VNode(4546.000, 0.000, 4545 | dict_keys([Action(0), Action(1), Action(2)]))} State((2, 0, 0, 0, 0, 0, 0, 1, 0), 0, 0) Observation((0, 0, 0, 0, 0, 1))
==== Step 2 ====
True state: State((2, 0, 0, 0, 0, 0, 0, 1, 0), 0, 0), 2 0
Action: Action(2)
Observation: Observation((0, 0, 0, 0, 0, 2)),  2
Reward: 0.0
True next state: State((2, 0, 0, 1, 0, 0, 0, 2, 0), 1, 0), 3 0
{Observation((0, 0, 0, 0, 0, 2)): VNode(5893.000, 0.000, 4840 | dict_keys([Action(0), Action(1), Action(2)])), Observation((0, 0, 0, 0, 1, 1)): VNode(3885.000, 0.000, 3185 | dict_keys([Action(0), Act

In [4]:
for a in range(P.n):
    print(card_problem.agent.tree[a], card_problem.agent.tree)
    print(tree_i[a])

QNode(13621.000, -2.924 | dict_keys([Observation((0, 1, 0, 0, 0, 0)), Observation((1, 0, 0, 0, 0, 0))])) VNode(40000.000, -2.920 | dict_keys([Action(0), Action(1), Action(2)]))
QNode(13621.000, -2.924 | dict_keys([Observation((0, 1, 0, 0, 0, 0)), Observation((1, 0, 0, 0, 0, 0))]))
QNode(12578.000, -2.946 | dict_keys([Observation((0, 0, 0, 1, 0, 0)), Observation((0, 0, 1, 0, 0, 0))])) VNode(40000.000, -2.920 | dict_keys([Action(0), Action(1), Action(2)]))
QNode(12578.000, -2.946 | dict_keys([Observation((0, 0, 0, 1, 0, 0)), Observation((0, 0, 1, 0, 0, 0))]))
QNode(13803.000, -2.920 | dict_keys([Observation((0, 0, 0, 0, 1, 0)), Observation((0, 0, 0, 0, 0, 1))])) VNode(40000.000, -2.920 | dict_keys([Action(0), Action(1), Action(2)]))
QNode(13803.000, -2.920 | dict_keys([Observation((0, 0, 0, 0, 1, 0)), Observation((0, 0, 0, 0, 0, 1))]))


In [5]:
planner = pomcp

s0 = env_reset_s0(card_problem)
print("s0 ",   s0)
total_reward = 0
action = planner.plan(card_problem.agent)
#action = policy.sample(card_problem.agent.cur_belief)

true_state = copy.deepcopy(card_problem.env.state)
env_reward = card_problem.env.state_transition(action, execute=True)
true_next_state = copy.deepcopy(card_problem.env.state)
real_observation = card_problem.env.provide_observation(card_problem.agent.observation_model, action)
card_problem.agent.update_history(action, real_observation)    
    
    
print("True state: %s, %d %d" % (true_state, np.array(true_state.val)[::3].sum(), int(true_state.terminal)))
print("Action: %s" % str(action))
print("Observation: %s,  %d" % (str(real_observation), np.array(real_observation.val).sum()))
print("Reward: %s" % str(np.maximum(0, env_reward)))
print("True next state: %s, %d %d" % (true_next_state, 
    np.array(true_next_state.val)[::3].sum(), int(true_next_state.terminal)))

env_reward = card_problem.env.state_transition(action, execute=True)
true_next_state = copy.deepcopy(card_problem.env.state)
real_observation = card_problem.env.provide_observation(card_problem.agent.observation_model, action)
card_problem.agent.update_history(action, real_observation)    
    
    
print("True state: %s, %d %d" % (true_state, np.array(true_state.val)[::3].sum(), int(true_state.terminal)))
print("Action: %s" % str(action))
print("Observation: %s,  %d" % (str(real_observation), np.array(real_observation.val).sum()))
print("Reward: %s" % str(np.maximum(0, env_reward)))
print("True next state: %s, %d %d" % (true_next_state, 
    np.array(true_next_state.val)[::3].sum(), int(true_next_state.terminal)))

s0  State((0, 0, 0, 1, 0, 0, 0, 0, 0), 1, None)
True state: State((0, 0, 0, 1, 0, 0, 0, 0, 0), 1, None), 1 0
Action: Action(2)
Observation: Observation((0, 0, 0, 0, 0, 1)),  1
Reward: 0.0
True next state: State((1, 0, 0, 1, 0, 0, 0, 1, 0), 0, 0), 2 0
True state: State((0, 0, 0, 1, 0, 0, 0, 0, 0), 1, None), 1 0
Action: Action(2)
Observation: Observation((0, 0, 0, 0, 0, 2)),  2
Reward: 0.0
True next state: State((1, 0, 0, 1, 0, 0, 1, 2, 0), 2, 0), 3 0


In [6]:
s0 = State(tuple(np.zeros(3*P.n, np.int32)), -1)
all_states = set()
create_all_states(s0, all_states)

In [7]:
(P.n*2)**(P.n*P.m)

46656

In [8]:
len(all_states)

909

In [9]:
# states for n=5;m=2 is 330438

In [10]:
for s in all_states:
    val = np.array(s.val)
    if (val[::3].sum()-1 != val[1::3].sum()) and s.card != "$" :
        print(s, val[::3].sum(), val[1::3].sum(),"error in states actions")

In [11]:
all_obs = create_all_observations(all_states)

In [12]:
len(all_obs)

539

In [18]:
rewards_all = np.load("rewards_pomcp_term.npy")

num = (rewards_all != 0).sum()
print(num)

print(rewards_all.sum() * 1.0 / num)

1
2.0


In [26]:
rewards_all = np.load("rewards_pouct_term.npy")

num = (rewards_all != 0).sum()
print(num)

print(rewards_all.sum() * 1.0 / num)

35
1.6857142857142857


In [21]:
card_problem.agent.tree.children

{Action(0): QNode(20422.000, -4.263 | dict_keys([Observation((0, 1, 0, 0, 0, 0)), Observation((1, 0, 0, 0, 0, 0)), Observation((0, 1, 0, 0, 1, 4))])),
 Action(1): QNode(18892.000, -4.282 | dict_keys([Observation((0, 0, 0, 1, 0, 0)), Observation((0, 0, 1, 0, 0, 0)), Observation((0, 1, 0, 0, 1, 4))])),
 Action(2): QNode(20688.000, -4.260 | dict_keys([Observation((0, 0, 0, 0, 1, 0)), Observation((0, 0, 0, 0, 0, 1)), Observation((0, 1, 0, 0, 1, 4))]))}

In [15]:
rewards_all

array([3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [16]:
from julia.api import Julia
jl = Julia(compiled_modules=False)

from julia.QMDP import QMDPSolver
from julia.SARSOP import SARSOPSolver

ModuleNotFoundError: No module named 'julia'